### 1.1. Getting pretrained model from torchvision

- ConvNet
- AlexNet
- DenseNet
- EfficientNet
- Google LeNet
- Inception
- ResNet
- SqueezeNet
- VGG

Lightweights:
- TinyVGG

HeavyWeights:
- VIT

In [ ]:
import torch
import torchvision
import torchinfo

In [ ]:
device = 'mps'

In [ ]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

In [ ]:
print(model)

In [ ]:
torchinfo.summary(
    model=model,
    input_size=(32, 3, 224, 224), 
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"]
)

In [ ]:
for layer in model.named_modules():
    print(layer)

### 1.2. Freeze encoder base for fine-tuning

In [ ]:
for param in model.features.parameters():
    param.requires_grad = False

In [ ]:
torchinfo.summary(
    model=model,
    input_size=(32, 3, 224, 224),
    col_names=["input_size", "output_size", "num_params", "trainable"],
)

### 1.3. Change Classifier

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [ ]:
output_shape = 3

model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1280, out_features=output_shape, bias=True), 
).to(device)

In [ ]:
torchinfo.summary(
    model=model,
    input_size=(32, 3, 224, 224),
    col_names=["input_size", "output_size", "num_params", "trainable"],
)

In [ ]:
from training.runners.torch import TorchTrainerModule

cfg = {
    'num_epochs': 3, 
    'device': 'mps',
    'log_every_n_steps': 1
}

trainer = TorchTrainerModule(cfg)

In [ ]:
from timeit import default_timer as timer 
start_time = timer

# trainer.fit()

end_time = timer()

### Getting pre-trained model from timm

In [ ]:
import timm

In [ ]:
# 'resnet18' in timm.list_models()

In [ ]:
# timm.list_models('resnet*')

In [ ]:
model2 = timm.create_model('resnet18', pretrained=True)
model3 = timm.create_model('resnet18', pretrained=True, features_only=True)

In [ ]:
# timm.list_models('efficient*')
model4 = timm.create_model('efficientnet_b0', pretrained=True, features_only=True)

In [ ]:
torchinfo.summary(model4, 
                  input_size=(32, 3, 224, 224),
                  col_names=["input_size", "output_size", "num_params", "trainable"])

In [ ]:
# model3

In [ ]:
torchinfo.summary(model3, 
                  input_size=(32, 3, 224, 224),
                  col_names=["input_size", "output_size", "num_params", "trainable"])

In [ ]:
model2